In [1]:
# Dependencies
import requests
from config import api_key, CLIENT_ID, CLIENT_SECRET
import json
import pprint as pp
import pandas as pd
import time
import lyricsgenius

In [2]:
# Request Access Token for Spotify API calls.
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [3]:
# Build query URL
# Access the playlist here: https://open.spotify.com/playlist/1Jczb1SckWxYiqrIzv0jsl?si=560c28541ccb42f5

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

url = "https://api.spotify.com/v1/"

play_id = "1Jczb1SckWxYiqrIzv0jsl?si=560c28541ccb42f5"

query_url = url+'playlists/'+play_id+'tracks'

In [4]:
# Get playlist data and initialize arrays for DataFrame assembly.
json_req = requests.get(query_url, headers=headers)
response = json_req.json()

track_ids = []
track_names = []
artist_names = []
acousticness = []
danceability = []
energy = []
instrumentalness = []
liveness = []
loudness = []
speechiness = []
tempo = []
valence = []

#Assign each genre as listed in the playlist.
genres = []
for j in range (1,9):
    for i in range(1,16):
        genres.append(j)

In [5]:
#Append track ID and related song data, then make separate API call to find the track's audio features.
first_part = True
while True:
    if first_part == True:
        for item in response['tracks']['items']:
            track_names.append(item['track']['name'])
            track_ids.append(item['track']['id']) 
            artist_names.append(item['track']['album']['artists'][0]['name'])
            #Make new API call on track's audio features based on each track's ID.
            audio_query_url = url+'audio-features/'+item['track']['id']
            json_req = requests.get(audio_query_url, headers=headers)
            audio_response = json_req.json()
            acousticness.append(audio_response['acousticness'])
            danceability.append(audio_response['danceability'])
            energy.append(audio_response['energy'])
            instrumentalness.append(audio_response['instrumentalness'])
            liveness.append(audio_response['liveness'])
            loudness.append(audio_response['loudness'])
            speechiness.append(audio_response['speechiness'])
            tempo.append(audio_response['tempo'])
            valence.append(audio_response['valence'])
        #Switch to second half of API request (Index max of 100 tracks in playlist per request)
        audio_response = response['tracks']['next']
        first_part = False
        time.sleep(3)
    else:
        req = requests.get(audio_response, headers=headers)
        data = req.json()
        for item in data['items']:
            track_names.append(item['track']['name'])
            track_ids.append(item['track']['id']) 
            artist_names.append(item['track']['album']['artists'][0]['name'])
            #Make new API call on track's audio features based on each track's ID.
            audio_query_url = url+'audio-features/'+item['track']['id']
            json_req = requests.get(audio_query_url, headers=headers)
            audio_response = json_req.json()
            acousticness.append(audio_response['acousticness'])
            danceability.append(audio_response['danceability'])
            energy.append(audio_response['energy'])
            instrumentalness.append(audio_response['instrumentalness'])
            liveness.append(audio_response['liveness'])
            loudness.append(audio_response['loudness'])
            speechiness.append(audio_response['speechiness'])
            tempo.append(audio_response['tempo'])
            valence.append(audio_response['valence'])
        break
#Assemble the DataFrame.
df = pd.DataFrame({ 'Track Name':track_names, 'Artist':artist_names, 'Track ID':track_ids, 'Genres':genres, 'Acousticness':acousticness,
                  'Danceability':danceability, 'Energy':energy, 'Instrumentalness':instrumentalness, 'Liveness':liveness,'Loudness':loudness,
                  'Speechiness':speechiness, 'Tempo':tempo, 'Valence':valence})



In [6]:
df.head(20)

,Track Name,Artist,Track ID,Genres,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence
0,The Real Slim Shady,Eminem,3yfqSUWxFvZELEM4PmlwIR,1,0.030200,0.949,0.661,0.000000,0.0454,-4.244,0.0572,104.504,0.760
1,MotorSport,Migos,4wFjTWCunQFKtukqrNijEt,1,0.030500,0.904,0.518,0.000000,0.3250,-5.320,0.1830,137.996,0.188
2,In Da Club,50 Cent,5Mcvgt2GbxxCp6cz3Kfjey,1,0.325000,0.906,0.663,0.000000,0.0883,-3.955,0.3550,90.042,0.847
3,Gangsta's Paradise,Coolio,1DIXPcTDzTj8ZMHt3PDt8p,1,0.065500,0.647,0.514,0.000000,0.3980,-10.050,0.0593,79.975,0.386
4,rockstar,Post Malone,19bX4zmC2zDben2ldhoRB3,1,0.145000,0.594,0.507,0.000871,0.1320,-6.141,0.0816,159.729,0.137
5,HUMBLE.,Kendrick Lamar,131OLY5J8XyfGuSjXRiTRM,1,0.000253,0.911,0.566,0.000075,0.0956,-6.703,0.1350,150.039,0.374
6,Black and Yellow,Wiz Khalifa,3VP78k3jzm0Q5OM08E383k,1,0.061900,0.685,0.832,0.000000,0.2730,-4.548,0.0595,164.070,0.574
7,WAP (feat. Megan Thee Stallion),Cardi B,051wt8AyLFgYnVuberd3vO,1,0.027200,0.934,0.443,0.000000,0.0889,-7.541,0.4100,133.026,0.359
8,Panini,Lil Nas X,6fTt0CH2t0mdeB2N9XFG5r,1,0.342000,0.703,0.594,0.000000,0.1230,-6.146,0.0752,153.848,0.475
9,Reflections,Bliss n Eso,5Wf0EKbK4AB6xtSbsqL4r2,1,0.725000,0.497,0.605,0.000000,0.2380,-8.625,0.0468,77.811,0.824


In [7]:
#Append lyric information from Genius API.
#Import ID and Secret.
CLIENT_ID = 'mVxsgubdxvPwivQOYFWt5FCskAPVyDRl7ZQnt4PPvlBaPHfbTATSqoqt2KKVjji1'
CLIENT_SECRET = 'u6bH4rznsJK0A80jHqIkJ3I6XxJr6IbtY0EX7T4aDDre5wCREj9ywuA5khlbyNM9AeKLoSFcYjYLA6vCTOKhPg'
genius = lyricsgenius.Genius('3Ee9v1MsjFxlAwwA0OkZWNJEHWRBhYWqpp5TwwWk_UX70_P36TH7StPKeBLt88EI')

In [8]:
#For loop that iterates through first column of df, searches for 
lyrics = []
failed = []
i = 0
for index, row in df.iterrows():
    song_title = row['Track Name']
    try:
        songs = genius.search_song(song_title)
        lyrics.append(songs.lyrics)
        time.sleep(2)
    except:
        try:
            songs = genius.search_song(song_title)
            lyrics.append(songs.lyrics)
            time.sleep(2)
        except:
            lyrics.append(0)
            print('Error retrieving lyrics. Retrying on next track...')
            failed.append(song_title)
            time.sleep(5)
            pass
pp.pprint(lyrics)

Searching for "The Real Slim Shady"...
Done.
Searching for "MotorSport"...
Done.
Searching for "In Da Club"...
Searching for "In Da Club"...
Error retrieving lyrics. Retrying on next track...
Searching for "Gangsta's Paradise"...
Searching for "Gangsta's Paradise"...
Error retrieving lyrics. Retrying on next track...
Searching for "rockstar"...
Done.
Searching for "HUMBLE."...
Done.
Searching for "Black and Yellow"...
Searching for "Black and Yellow"...
Error retrieving lyrics. Retrying on next track...
Searching for "WAP (feat. Megan Thee Stallion)"...
Done.
Searching for "Panini"...
Done.
Searching for "Reflections"...
Done.
Searching for "Push It"...
Done.
Searching for "Bodak Yellow"...
Searching for "Bodak Yellow"...
Done.
Searching for "Super Bass"...
Done.
Searching for "Paper Planes"...
Searching for "Paper Planes"...
Error retrieving lyrics. Retrying on next track...
Searching for "Black Widow"...
Done.
Searching for "Rapper's Delight"...
Done.
Searching for "Straight Outta Co

 'Then your six-inch heels, catch her\n'
 "She's headed to the pyramid\n"
 '[Chorus]\n'
 "She's working at the pyramid tonight\n"
 'Working at the pyramid\n'
 'Working at the pyramid tonight\n'
 'Working at the pyramid\n'
 'Working at the pyramid tonight\n'
 'Working at the pyramid\n'
 'Working at the pyramid tonight\n'
 'Working at the pyramid\n'
 'Working at the pyramid tonight\n'
 '\n'
 '[Verse 1]\n'
 'Pimping in my convos\n'
 'Bubbles in my champagne, let it be some jazz playing\n'
 'Top floor motel suite twisting my cigars\n'
 'Floor model TV with the VCR\n'
 'Got rubies in my damn chain\n'
 "Whip ain't got no gas tank but it still got woodgrain\n"
 'Got your girl working for me\n'
 'Hit the strip and my bills paid\n'
 'That keep my bills paid\n'
 'Hit the strip and my bills paid\n'
 'Keep a nigga bills paid\n'
 '\n'
 '[Chorus]\n'
 "She's working at the pyramid tonight\n"
 'Working at the pyramid\n'
 'Working at the pyramid tonight\n'
 'Working at the pyramid\n'
 'Working at the p

 "7 ContributorsAngela’s Ashes LyricsAngela's Ashes\n"
 '\n'
 'A Memoir of a Childhood\n'
 '\n'
 'By Frank McCourt\n'
 '\n'
 'This book is dedicated to my brothers, Malachy, Michael, Alphonsus. I learn '
 'from you, I admire you and I love you.\n'
 '\n'
 'Acknowledgments\n'
 '\n'
 'This is a small hymn to an exaltation of women.\n'
 "R'lene Dahlberg fanned the embers.\n"
 'Lisa Schwarzbaum read early pages and encouraged me.\n'
 'Mary Breasted Smyth, elegant novelist herself, read the first third\n'
 'and passed it on to Molly Friedrich, who became my agent and thought that '
 'Nan Graham, Editor-in-Chief at Scribner, would be just the right person to '
 'put the book on the road.\n'
 'And Molly was right.\n'
 'My daughter, Maggie, has shown me how life can be a grand adventure, while '
 'exquisite moments with my granddaughter, Chiara, have helped me recall a '
 "small child's wonder.\n"
 'My wife, Ellen, listened while I read and cheered me to the final\n'
 'page.\n'
 'I am blessed a

 "barely two and already playing with his little brother in heaven? You'll "
 "take your son with you and he'll remind you this is no day for the pubs. She "
 'looks at him and he looks at her and he puts on his cap.\n'
 'At the Labour Exchange we stand at the end of the queue till a man comes '
 "from behind the counter and tells Dad he's very sorry for his troubles and "
 'he should go ahead of everyone else on this sorrowful day. Men touch their '
 "caps and say they're sorry for his troubles and some pat my head and give me "
 "pennies, twenty-four pennies, two shillings. Dad tells me I'm rich now and I "
 'should buy myself a sweet while he goes into this place for a minute. I know '
 'this place is a pub and I know he wants to get the black stuff that is '
 "called a pint but I don't say anything because I want to go to the shop next "
 'door for a piece of toffee. I chew my toffee till it melts and leaves my '
 'mouth all sweet and sticky. Dad is still in the pub and I wonder if

 "squatting and saying, There's nothing in the world like a good feed of "
 'apples, a drink of water and a good shit, better than any sangwidge of '
 "cheese and mustard and Dotty O'Neill can shove his apple up his arse.\n"
 'There are three cows in a field with their heads over a stone wall and they '
 "say moo to us. Paddy says, Bejasus, 'tis milkin' time, and he's over the "
 'wall, stretched on his back under a cow with her big udder hanging into his '
 'face. He pulls on a teat and squirts milk into his mouth. He stops squirting '
 "and says, Come on, Frankie, fresh milk. 'Tis lovely. Get that other cow, "
 "they're all ready for the milkin'.\n"
 "I get under the cow and pull on a teat but she kicks and moves and I'm sure "
 "she's going to kill me. Paddy comes over and shows me how to do it, pull "
 'hard and straight and the milk comes out in a powerful stream. The two of us '
 "lie under the one cow and we're having a great time filling ourselves with "
 "milk when there's a r

 "and why couldn't God take the rich for a change and is there any lemonade in "
 'the house? Michael wants to know if Mam will die and Malachy tells him you '
 "can't die till a priest comes. Then Michael wonders if we'll ever have a "
 "fire and hot tea again because he's freezing in the bed even with the "
 'overcoats left over from\n'
 'olden times. Malachy says we should go from house to house asking for turf '
 "and coal and wood and we could use Alphie's pram to carry the load. We "
 "should take Alphie with us because he's small and he smiles and people will "
 'see him and feel sorry for him and us. We try to wash all the dirt and lint '
 'and feathers and sticky marmalade but when we touch him with water he howls. '
 "Michael says he'll only get dirty again in the pram so what's the use of "
 "washing him. Michael is small but he's always saying remarkable things like "
 'that.\n'
 'We push the pram out to the rich avenues and roads but when we knock on the '
 "doors the maid

 'Mr. Carmody snuffles and looks puzzled. The priest recites the Latin prayers '
 'and sprinkles the coffin with holy water.\n'
 'I want to go to the priest, to Mr. and Mrs. Carmody. I want to tell them how '
 "I'm the one who sent Theresa to hell. They can do whatever they like with "
 'me. Abuse me. Revile me. Throw grave dirt at me. But I stay behind the tree '
 'till the mourners leave and the grave diggers fill in the grave.\n'
 'Frost is already whitening the fresh earth on the grave and I think of '
 "Theresa cold in the coffin, the red hair, the green eyes. I can't understand "
 'the feelings going through me but I know that with all the people who died '
 'in my family and all the people who died in the lanes around me and all the '
 'people who left I never had a pain like this in my heart and I hope I never '
 'will again.\n'
 "It's getting dark. I walk my bicycle out of the graveyard. I have telegrams "
 'to deliver.\n'
 'XVI\n'
 "Mrs. O'Connell gives me telegrams to delive

 'Nodfather — “Thoughts” (2018)\n'
 'Brett — “Therapy” (2018)\n'
 'Ra’Shaun — “She Told Me” (2018)\n'
 'S7EAZE — “Don’t Test Me” (2018)\n'
 'M.R — “Calling” (2018)\n'
 'Xander — “Better or Worse” (2018)\n'
 'Ric Wilson — BANBA (2018)\n'
 'Adamn Killa — Adamn Everlasting (In Loving Memory of Jalen) (2018)\n'
 'GNGR — “Without You” (2018)\n'
 'Star Martin — “Walls” (2018)\n'
 'Trevor Daniel — “Wake Up” (2018)\n'
 'BR3 (feat. Jacquees) — “Used to This” (2018)\n'
 'p-rallel (feat. Yiigaa & Finn Foxell) — “Signs” (2018)\n'
 'gny — Present (2018)\n'
 'Sameik Moore — “Bounce” (2018)\n'
 'Adreyn Cash — “Angel’s Lie” (2018)\n'
 'Parker Louis — All Good Things, Pt. One (2018)\n'
 'Tru — “Young & Reckless” (2018)\n'
 'Bae Louie — “Wonderful” (2018)\n'
 'Rokusho (feat. Inga & Stille) — “Wishing for You” (2018)\n'
 'bêlla — “sorry if u misunderstood” (2018)\n'
 'SVĒ — “Njne1 (Not Just Anyone)” (2018)\n'
 'Shaeane — “Bag of Bones” (2018)\n'
 'Shaeane — “Joker” (2018)\n'
 'Olloway — “August” (2018)\n

 '[COMPANY]\n'
 'Y si pierdes mis huellas que dios te bendiga\n'
 'Respira\n'
 '\n'
 '[NINA (COMPANY)]\n'
 'As the radio plays old forgotten boleros\n'
 'I think of the days when this city was mine\n'
 'I remember the praise\n'
 '“Ay, te adoro, te quiero” (Te adoro)\n'
 'The neighborhood waved\n'
 'And said, “Nina, be brave (Te quiero)\n'
 'And you’re gonna be fine!” (Respira)\n'
 'And maybe it’s me\n'
 'But it all seems like lifetimes ago (Respira)\n'
 'So what do I say to these faces that I used to know?\n'
 '“Hey, I’m home?”\n'
 '[WOMAN]\n'
 'Mira, Nina\n'
 '\n'
 '[NINA]\n'
 'Hey\n'
 '[COMPANY]\n'
 'No me preocupo por ella\n'
 '\n'
 '[NINA]\n'
 'They’re not worried about me\n'
 '\n'
 '[COMPANY]\n'
 'Mira, allí esta nuestra estrella\n'
 '\n'
 '[NINA]\n'
 'They are all counting on me to succeed\n'
 '\n'
 '[COMPANY]\n'
 'Ella sí da la talla\n'
 '\n'
 '[NINA (COMPANY)]\n'
 'I am the one who made it out (Ah)\n'
 'The one who always made the grade\n'
 'But maybe I should have just stayed 

In [11]:
#Append lyric information to df.

df['Lyrics'] = lyrics
df.head()

,Track Name,Artist,Track ID,Genres,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Lyrics
0,The Real Slim Shady,Eminem,3yfqSUWxFvZELEM4PmlwIR,1,0.0302,0.949,0.661,0.000000,0.0454,-4.244,0.0572,104.504,0.760,459 ContributorsTranslationsEspañolPortuguêsPo...
1,MotorSport,Migos,4wFjTWCunQFKtukqrNijEt,1,0.0305,0.904,0.518,0.000000,0.3250,-5.320,0.1830,137.996,0.188,428 ContributorsTranslationsPortuguêsMotorSpor...
2,In Da Club,50 Cent,5Mcvgt2GbxxCp6cz3Kfjey,1,0.3250,0.906,0.663,0.000000,0.0883,-3.955,0.3550,90.042,0.847,0
3,Gangsta's Paradise,Coolio,1DIXPcTDzTj8ZMHt3PDt8p,1,0.0655,0.647,0.514,0.000000,0.3980,-10.050,0.0593,79.975,0.386,0
4,rockstar,Post Malone,19bX4zmC2zDben2ldhoRB3,1,0.1450,0.594,0.507,0.000871,0.1320,-6.141,0.0816,159.729,0.137,467 ContributorsTranslationsEspañolPolskiFranç...


In [35]:
# Try again for empty rows. Run this section until all lyrics implemented.
lyrics = []
failed = []
i = 0
for row in df.index:
    if df['Lyrics'][row] == 0:
        song_title = df['Track Name'][row]
        try:
            songs = genius.search_song(song_title)
            df['Lyrics'][row] = songs.lyrics
            time.sleep(2)
        except:
            try:
                songs = genius.search_song(song_title)
                df['Lyrics'][row] = songs.lyrics
                time.sleep(2)
            except:
                lyrics.append(0)
                print('Error retrieving lyrics. Skipping to next track...')
                failed.append(song_title)
                time.sleep(5)
                pass

Searching for "Losing My Religion"...
Done.


C:\Users\Alex\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Searching for "Sweet Home Alabama"...
Done.


In [40]:
print(df.iloc[0,13])

459 ContributorsTranslationsEspañolPortuguêsPolskiItalianoDeutschFrançaisTürkçeThe Real Slim Shady Lyrics[Intro]
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here...

[Verse 1]
Y'all act like you never seen a white person before
Jaws all on the floor like Pam like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (Agh!)
It's the return of the "Oh, wait, no way, you're kidding
He didn't just say what I think he did, did he?"
And Dr. Dre said… nothing, you idiots!
Dr. Dre's dead, he's locked in my basement (Ha ha!)
Feminist women love Eminem
"Chicka, chicka, chicka, Slim Shady, I'm sick of him
Look at him, walkin' around, grabbin' his you-know-what
Flippin' the you-know-who," "Yeah, but he's so cute though"
Yeah, I probably got a couple of screws up in my head loose

In [41]:
df.head()

,Track Name,Artist,Track ID,Genres,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Lyrics
0,The Real Slim Shady,Eminem,3yfqSUWxFvZELEM4PmlwIR,1,0.0302,0.949,0.661,0.000000,0.0454,-4.244,0.0572,104.504,0.760,459 ContributorsTranslationsEspañolPortuguêsPo...
1,MotorSport,Migos,4wFjTWCunQFKtukqrNijEt,1,0.0305,0.904,0.518,0.000000,0.3250,-5.320,0.1830,137.996,0.188,428 ContributorsTranslationsPortuguêsMotorSpor...
2,In Da Club,50 Cent,5Mcvgt2GbxxCp6cz3Kfjey,1,0.3250,0.906,0.663,0.000000,0.0883,-3.955,0.3550,90.042,0.847,245 ContributorsTranslationsTürkçePortuguês50 ...
3,Gangsta's Paradise,Coolio,1DIXPcTDzTj8ZMHt3PDt8p,1,0.0655,0.647,0.514,0.000000,0.3980,-10.050,0.0593,79.975,0.386,234 ContributorsTranslationsMagyarGangsta’s Pa...
4,rockstar,Post Malone,19bX4zmC2zDben2ldhoRB3,1,0.1450,0.594,0.507,0.000871,0.1320,-6.141,0.0816,159.729,0.137,467 ContributorsTranslationsEspañolPolskiFranç...
